In [44]:
from datetime import datetime
start_time = datetime.now()

import pandas as pd
import xgboost as XGBClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [45]:
%%time
vars = pd.read_csv('vars_final.csv')
print(vars.shape)
vars.head()

(96397, 32)
Wall time: 345 ms


,card_merch_total_7,card_state_max_60,merch_state_total_0,merch_zip_max_7,Merchnum_max_7,card_merch_total_14,Merchnum_total_0,Merchnum_avg_7,card_merch_total_60,card_zip_total_30,...,state_des_max_3,card_state_avg_60,state_des_avg_1,merch_zip_med_1,merch_state_avg_1,Merchnum_avg_1,merch_zip_avg_14,state_des_total_0,Recnum,Fraud
0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,1,0
1,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,...,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,2,0
2,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,...,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,3,0
3,3.62,3.62,7.24,3.62,3.62,3.62,7.24,3.62,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,4,0
4,7.24,3.62,10.86,3.62,3.62,7.24,10.86,3.62,7.24,7.24,...,3.62,3.62,3.62,3.62,3.62,3.62,3.62,7.24,5,0


In [46]:
final_vars = pd.read_csv('final_vars_list.csv')
final_vars

,variable name
0,card_merch_total_7
1,card_state_max_60
2,merch_state_total_0
3,merch_zip_max_7
4,Merchnum_max_7
5,card_merch_total_14
6,Merchnum_total_0
7,Merchnum_avg_7
8,card_merch_total_60
9,card_zip_total_30


In [47]:
vars.rename(columns={'recnum':'Recnum'},inplace=True)
numvars = 10
final_vars_list = ['Recnum','Fraud']
for i in range(numvars):
    final_vars_list.append(final_vars.iloc[i]['variable name'])
    
final_vars_list

['Recnum',
 'Fraud',
 'card_merch_total_7',
 'card_state_max_60',
 'merch_state_total_0',
 'merch_zip_max_7',
 'Merchnum_max_7',
 'card_merch_total_14',
 'Merchnum_total_0',
 'Merchnum_avg_7',
 'card_merch_total_60',
 'card_zip_total_30']

In [48]:
vars = vars.filter(final_vars_list,axis=1)
vars.head()

,Recnum,Fraud,card_merch_total_7,card_state_max_60,merch_state_total_0,merch_zip_max_7,Merchnum_max_7,card_merch_total_14,Merchnum_total_0,Merchnum_avg_7,card_merch_total_60,card_zip_total_30
0,1,0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62
1,2,0,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42
2,3,0,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49
3,4,0,3.62,3.62,7.24,3.62,3.62,3.62,7.24,3.62,3.62,3.62
4,5,0,7.24,3.62,10.86,3.62,3.62,7.24,10.86,3.62,7.24,7.24


In [49]:
# use this to cap variables. For some problems it helps
Clip = 10

In [50]:
record_save = vars['Recnum']
Y_save = pd.DataFrame(vars.loc[:,'Fraud'])
Y_save.head()

,Fraud
0,0
1,0
2,0
3,0
4,0


In [51]:
## Scale and truncate field values

X_no_scaling = vars.drop(columns = ['Recnum','Fraud'])
X_no_scaling.describe()

X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()

# push in any outlier values
cols = X.columns
X.loc[:,cols] = X[cols].clip(upper=Clip)
X.loc[:,cols] = X[cols].clip(lower=-1*Clip)
# X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()
X.describe()

# separate data into modeling (traintest) and out of time
oot_recnum=84300
X_trntst = X[0:oot_recnum]
Y_trntst = Y_save[0:oot_recnum]
X_oot = X[oot_recnum:]
Y_oot = Y_save[oot_recnum:]

niter = 0
nitermax = 10
X_oot_orig = X_oot.copy()
pd.options.mode.chained_assignment = None  # default='warn'

In [52]:
Modeling_output = pd.DataFrame(columns=['Model','Trn','Tst','OOT'],index=range(1000))
counter = 0
model_counter = 0

In [54]:
# Catboost

FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
for niter in range(nitermax):  
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = CatBoostClassifier(verbose=0,
            iterations=1000,learning_rate=0.01,l2_leaf_reg=5,max_depth=7)
#

    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    Modeling_output.iloc[counter] = ['cat boost',FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot']]
    counter = counter + 1
    
print(FDR3.mean())
model_counter = model_counter + 1

0 0.8881578947368421 0.8345588235294118 0.5754189944134078
1 0.8947368421052632 0.8786764705882353 0.5810055865921788
2 0.8865131578947368 0.8382352941176471 0.5754189944134078
3 0.8901453957996769 0.8390804597701149 0.5754189944134078
4 0.8859375 0.8625 0.547486033519553
5 0.8747940691927513 0.8754578754578755 0.5754189944134078
6 0.8808777429467085 0.8471074380165289 0.5810055865921788
7 0.8861386138613861 0.8211678832116789 0.5810055865921788
8 0.878740157480315 0.8530612244897959 0.5810055865921788
9 0.8727858293075684 0.8841698841698842 0.5698324022346368
trn    0.883883
tst    0.853402
oot    0.574302
dtype: float64
